In [1]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="myfirstdb",
    user="postgres",
    password="524919",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Create tables
cur.execute("""
    CREATE TABLE IF NOT EXISTS Customers (
        customer_id SERIAL PRIMARY KEY,
        name VARCHAR(100),
        email VARCHAR(100) UNIQUE,
        phone VARCHAR(20),
        address TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS Products (
        product_id SERIAL PRIMARY KEY,
        name VARCHAR(100),
        category VARCHAR(50),
        price DECIMAL(10,2),
        stock_quantity INT
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS Orders (
        order_id SERIAL PRIMARY KEY,
        customer_id INT REFERENCES Customers(customer_id) ON DELETE CASCADE,
        order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        total_amount DECIMAL(10,2),
        status VARCHAR(20) CHECK (status IN ('Pending', 'Completed', 'Cancelled'))
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS OrderDetails (
        order_id INT REFERENCES Orders(order_id) ON DELETE CASCADE,
        product_id INT REFERENCES Products(product_id) ON DELETE CASCADE,
        quantity INT,
        subtotal DECIMAL(10,2),
        PRIMARY KEY (order_id, product_id)
    );
""")

conn.commit()
print("Tables created successfully!")

cur.close()
conn.close()


Tables created successfully!


In [2]:
import pandas as pd

# Load data
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
orders = pd.read_csv("orders.csv")
order_details = pd.read_csv("order_details.csv")

# Insert data into tables
conn = psycopg2.connect(
    dbname="myfirstdb",
    user="postgres",
    password="524919",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Function to insert data
def insert_data(table, dataframe):
    for _, row in dataframe.iterrows():
        values = tuple(row)
        placeholders = ", ".join(["%s"] * len(values))
        query = f"INSERT INTO {table} VALUES ({placeholders})"
        cur.execute(query, values)

insert_data("Customers", customers)
insert_data("Products", products)
insert_data("Orders", orders)
insert_data("OrderDetails", order_details)

conn.commit()
print("Data inserted successfully!")

cur.close()
conn.close()


Data inserted successfully!
